## Assembling a Feature Engineering Pipeline with Feature-engine and Cross-validation

We will:
- build a gradient boosted tree
- use feature-engine for the feature engineering steps
- set up an entire engineering and prediction pipeline using a Scikit-learn Pipeline
- train the pipeline with cross-validation, looking over different feature-engineering parameters

In [15]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# for the model
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import roc_auc_score
from sklearn.pipeline import Pipeline

# for feature engineering
from feature_engine import missing_data_imputers as mdi
from feature_engine import discretisers as dsc
from feature_engine import categorical_encoders as ce

In [16]:
# load dataset

cols = [
    'pclass', 'sex', 'age', 'sibsp', 'parch', 'fare', 'cabin',
    'embarked', 'survived'
]

data = pd.read_csv('titanic.csv', usecols=cols)

data.head()

,pclass,survived,sex,age,sibsp,parch,fare,cabin,embarked
0,1,1,female,29.0000,0,0,211.3375,B5,S
1,1,1,male,0.9167,1,2,151.5500,C22,S
2,1,0,female,2.0000,1,2,151.5500,C22,S
3,1,0,male,30.0000,1,2,151.5500,C22,S
4,1,0,female,25.0000,1,2,151.5500,C22,S


In [17]:
# Cabin: extract numerical and categorical part and delete original variable

data['cabin_num'] = data['cabin'].str.extract('(\d+)') # captures numerical part
data['cabin_num'] = data['cabin_num'].astype('float')
data['cabin_cat'] = data['cabin'].str[0] # captures the first letter

data.drop(['cabin'], axis=1, inplace=True)

In [18]:
# make list of variables types

# numerical: discrete and continuous
discrete = [
    var for var in data.columns if data[var].dtype != 'O' and var != 'survived'
    and data[var].nunique() < 10
]
continuous = [
    var for var in data.columns
    if data[var].dtype != 'O' and var != 'survived' and var not in discrete
]

# categorical
categorical = [var for var in data.columns if data[var].dtype == 'O']

print('There are {} discrete variables'.format(len(discrete)))
print('There are {} continuous variables'.format(len(continuous)))
print('There are {} categorical variables'.format(len(categorical)))

There are 3 discrete variables
There are 3 continuous variables
There are 3 categorical variables


In [19]:
# separate into training and testing set

X_train, X_test, y_train, y_test = train_test_split(
    data.drop('survived', axis=1),  # predictors
    data['survived'],  # target
    test_size=0.1,  # percentage of obs in test set
    random_state=0)  # seed to ensure reproducibility

X_train.shape, X_test.shape

((1178, 9), (131, 9))

### Set up the pipeline

In [20]:
titanic_pipe = Pipeline([

    # missing data imputation
    ('imputer_num',
     mdi.ArbitraryNumberImputer(arbitrary_number=-1,
                                variables=['age', 'fare', 'cabin_num'])),
    ('imputer_cat',
     mdi.CategoricalVariableImputer(variables=['embarked', 'cabin_cat'])),

    # categorical encoding
    ('encoder_rare_label',
     ce.RareLabelCategoricalEncoder(tol=0.01,
                                    n_categories=2,
                                    variables=['embarked', 'cabin_cat'])),
    ('categorical_encoder',
     ce.OrdinalCategoricalEncoder(encoding_method='ordered',
                                  variables=['cabin_cat', 'sex', 'embarked'])),

    # Gradient Boosted machine
    ('gbm', GradientBoostingClassifier(random_state=0))
])

## Set up the Grid ad the cross-validation strategy

In [21]:
# now we create the grid with all the parameters that we would like to test

param_grid = {
    # try different feature engineering parameters
    'imputer_num__arbitrary_number': [-1, 99],
    'encoder_rare_label__tol': [0.1, 0.2],
    'categorical_encoder__encoding_method': ['ordered', 'arbitrary'],
    
    # try different gradient boosted tree model paramenters
    'gbm__max_depth': [None, 1, 3],
}


# now we set up the grid search with cross-validation
grid_search = GridSearchCV(titanic_pipe, param_grid,
                           cv=5, iid=False, n_jobs=-1, scoring='roc_auc')

In [22]:
# and now we train over all the possible combinations of the parameters above
grid_search.fit(X_train, y_train)

# and we print the best score over the train set
print(("best roc-auc from grid search: %.3f"
       % grid_search.score(X_train, y_train)))

best roc-auc from grid search: 0.930


In [23]:
# we can print the best estimator parameters like this
grid_search.best_estimator_

Pipeline(memory=None,
     steps=[('imputer_num', ArbitraryNumberImputer(arbitrary_number=99,
            variables=['age', 'fare', 'cabin_num'])), ('imputer_cat', CategoricalVariableImputer(variables=['embarked', 'cabin_cat'])), ('encoder_rare_label', RareLabelCategoricalEncoder(n_categories=2, tol=0.1,
              variabl...    subsample=1.0, tol=0.0001, validation_fraction=0.1,
              verbose=0, warm_start=False))])

In [24]:
# and find the best fit parameters like this
grid_search.best_params_

{'categorical_encoder__encoding_method': 'ordered',
 'encoder_rare_label__tol': 0.1,
 'gbm__max_depth': 3,
 'imputer_num__arbitrary_number': 99}

In [25]:
# here we can see all the combinations evaluated during the gridsearch
grid_search.cv_results_['params']

[{'categorical_encoder__encoding_method': 'ordered',
  'encoder_rare_label__tol': 0.1,
  'gbm__max_depth': None,
  'imputer_num__arbitrary_number': -1},
 {'categorical_encoder__encoding_method': 'ordered',
  'encoder_rare_label__tol': 0.1,
  'gbm__max_depth': None,
  'imputer_num__arbitrary_number': 99},
 {'categorical_encoder__encoding_method': 'ordered',
  'encoder_rare_label__tol': 0.1,
  'gbm__max_depth': 1,
  'imputer_num__arbitrary_number': -1},
 {'categorical_encoder__encoding_method': 'ordered',
  'encoder_rare_label__tol': 0.1,
  'gbm__max_depth': 1,
  'imputer_num__arbitrary_number': 99},
 {'categorical_encoder__encoding_method': 'ordered',
  'encoder_rare_label__tol': 0.1,
  'gbm__max_depth': 3,
  'imputer_num__arbitrary_number': -1},
 {'categorical_encoder__encoding_method': 'ordered',
  'encoder_rare_label__tol': 0.1,
  'gbm__max_depth': 3,
  'imputer_num__arbitrary_number': 99},
 {'categorical_encoder__encoding_method': 'ordered',
  'encoder_rare_label__tol': 0.2,
  'gbm_

In [26]:
# and here the scores for each of one of the above combinations
grid_search.cv_results_['mean_test_score']

array([0.82941923, 0.82854885, 0.84155143, 0.84542691, 0.86234449,
       0.86825992, 0.82958482, 0.83174771, 0.84221378, 0.84529895,
       0.8626004 , 0.86762015, 0.82748442, 0.82424855, 0.84168279,
       0.84558796, 0.86236819, 0.86792228, 0.82861343, 0.8276732 ,
       0.84240535, 0.84546753, 0.86230045, 0.86772659])

In [27]:
# and finally let's check the performance over the test set
print(("best linear regression from grid search: %.3f"
       % grid_search.score(X_test, y_test)))

best linear regression from grid search: 0.848
